In [1]:
import cv2
import os

In [1]:
import cv2
import os

# 创建一个目录来保存采集到的图像
if not os.path.exists('dataset'):
    os.makedirs('dataset')

# 打开摄像头
cap = cv2.VideoCapture(0)
# face_detector = cv2.CascadeClassifier(cv2.data.haarcascades + 'Frontal Face Cascade.xml')
face_detector = cv2.CascadeClassifier('./Frontal Face Cascade.xml')

# 如果上面的路径有问题，可以改为绝对路径，例如：
# face_detector = cv2.CascadeClassifier('/path/to/haarcascade_frontalface_default.xml')

# 采集100张脸部图像
count = 0
while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(gray, 1.3, 5)
    
    for (x, y, w, h) in faces:
        count += 1
        # 保存采集到的图像到dataset目录
        cv2.imwrite(f"dataset/User.{count}.jpg", gray[y:y+h, x:x+w])
        
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv2.putText(frame, f"Image Count: {count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    cv2.imshow('image', frame)
    
    k = cv2.waitKey(100) & 0xff
    if k == 27 or count >= 200:  # 按下ESC键或采集到100张图像时退出
        break

cap.release()
cv2.destroyAllWindows()


[ WARN:0@0.015] global /private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_11nitadzeg/croot/opencv-suite_1691620374638/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


: 

In [2]:
#训练数据
import cv2
import numpy as np
from PIL import Image
import os

# 数据路径
path = 'dataset'
recognizer = cv2.face.LBPHFaceRecognizer_create()
detector = cv2.CascadeClassifier('./Frontal Face Cascade.xml')

# 从采集到的图像中获取训练数据
def get_images_and_labels(path):
    image_paths = [os.path.join(path, f) for f in os.listdir(path)]
    face_samples = []
    ids = []
    for image_path in image_paths:
        gray_img = Image.open(image_path).convert('L')
        img_np = np.array(gray_img, 'uint8')
        faces = detector.detectMultiScale(img_np)
        for (x, y, w, h) in faces:
            face_samples.append(img_np[y:y+h, x:x+w])
            ids.append(1)  # 这里使用1作为ID，因为这里只有一个用户
    return face_samples, ids

faces, ids = get_images_and_labels(path)
recognizer.train(faces, np.array(ids))

# 保存模型
if not os.path.exists('trainer'):
    os.makedirs('trainer')
recognizer.write('trainer/trainer.yml')

# 删除数据集图片的函数
def delete_images_in_folder(folder_path):
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                delete_images_in_folder(file_path)
        except Exception as e:
            print(f'Failed to delete {file_path}. Reason: {e}')

# 删除训练数据集和验证数据集图片
delete_images_in_folder('dataset')

In [3]:
import cv2

recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('trainer/trainer.yml')
face_cascade = cv2.CascadeClassifier('./Frontal Face Cascade.xml')

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        id_, confidence = recognizer.predict(roi_gray)
        
        if confidence < 100:
            confidence_text = f"  {round(100 - confidence)}%"
            cv2.putText(frame, "Recognized", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        else:
            confidence_text = f"  {round(100 - confidence)}%"
            cv2.putText(frame, "Unknown", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv2.putText(frame, confidence_text, (x + 5, y + h - 5), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    cv2.imshow('frame', frame)
    
    k = cv2.waitKey(10) & 0xff
    if k == 27:  # 按下ESC键退出
        break

cap.release()
cv2.destroyAllWindows()


[ WARN:0@21.926] global /private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_11nitadzeg/croot/opencv-suite_1691620374638/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


: 